In [245]:
# CCs
# ## In general
# * What is the role of the notebooks?
#   The preprocessing functions are defined in the notebook but they cannot easily be used as their application
#   is hidden away in `apply_modification`. This utility function is a good idea but as the notebook looks now, 
#   there is little benefit in putting all preprocessing code here. It could be in a separate module as well.
#   IMHO the strength of the notebook is that you can play around. Do you want that?
# * I believe it would be helpful to see before and after comparisons. The data set is too large to be easily 
#   inspected visually. Why not define a small subset to showcase the transformations?

# ## With regards to implementation
# * Most operations are quite straightforward but the code seems more complicated than necessary.
# * Preprocessing here is a pipeline involving several functions. These functions all modify their input. 
#   I believe it would be easier to showcase their behavior if they would not do that.
# * I would take more care of data types and defer more processing to the csv reader.
# * Feeling. The implementation makes me uneasy. Possible errors can pass silently (in variable selection steps) 
#   and therefore you can easily have ill-defined code without knowing it.
# * I would be more explicit with regards to data types.
# * Thought. Interesting to see how processing often depends on the wave but you infer the variable name using 
#   DataFrame.filter. 
# * It can be helpful to standardize your wording to make it easier to understand 
# the type of a manipulation. See e.g. https://dplyr.tidyverse.org/#overview
# * Since the notebook is already kind of lengthy, maybe split it into tidying and feature engineering?
# * I can imagine that preprocessing was accompanied by data exploration/visualization. The story may be 
#   more interesting with a few pictures and tables in between - to make the data tangible.   

# ## Observed patterns
# * Naming. On all levels, i.e. function names, docstrings, variables. I believe
#   in many cases naming/wording can be improved.
# * Proper use of constructors. Pass attributes during instantiation and not 
#   after. This makes the code easier to read.
# * Proper use of duck typing. Pass arguments, e.g. as interables instead of 
#   as lists and thereby save superfluous statements and increase readability.
# * A lot of functions look like this sandwich:
#   1. Find colum name or names
#   2. Do something with these variables
#   3. Concatenate output to input or modify in place
#   I would separate these steps to factor our boilerplate.
# * Column names. They are very important and a lot of duplication exists.
#   I think I would provide utilities just to handle names and be later able 
#   to perform consistency checks.

# Preprocessing

In [56]:
import sys
sys.path.insert(0, 'utils/')  # CC: Why? Put __init__.py into utils/ and then just import from util
from dictionaries_rename import *  # CC: Why? Avoid star import. Edit: This import stole me a minute figuring out a trivial function below. :P
from scipy.stats import rankdata
import pandas as pd
import numpy as np
import csv, warnings, string  # CC: Unused imports?
warnings.filterwarnings('ignore')

In [243]:
df = pd.read_csv('data/10017_da_en_v2_0.tab', sep='\t')

## Different features need to be processed differently

We split all features into groups that need to be processed differently:

A) X_personal: related to personal data (such as gender, age, etc) - what we could filter before the survey 

B) Related to questions about politics and their opinions:
   * X_dummies: categorical features that need to be one hot encoded and renamed (e.g. whom did they vote);
   * X_ordinal: ordinal features that need to be coded and renamed (e.g. strongly agree/agree/disagree/strongly disagree);
   * X_changed_names: ordinal features that need only to be renamed (e.g. probability to vote: already from 0 to 10);
   * X_binary: e.g. used facebook, mentioned particular question (yes/no)
   * engineered features

Predicting only 1 wave based on only previous one.

Each function (almost) is executed just after it was defined so it is easier to follow

## General preparation of data and functions used

filtering only rows from particular wave, transforming dte for waves 5 and 6 into negative since survey happened after elections

In [59]:
waves = ['1', '2', '3', '4', '5', '6']

In [60]:
df.replace({'refused': np.NaN, '': np.NaN}, inplace=True)  # CC: refused exists?
df = df.apply(pd.to_numeric, errors='ignore')  # CC: Would be more explicit and do it on read.

Making `dte` (="days to election") in wave 5 and 6 negative as they took place after the elections.

In [61]:
df[['w5_dte', 'w6_dte']] = df[['w5_dte', 'w6_dte']]*(-1)  # CC: Why not just df[['w5_dte', 'w6_dte']] = -df[['w5_dte', 'w6_dte']]? (Easier to read than *=-1)

In [62]:
def apply_modification(function, df_dict):  # CC: Naming.
    # CC: toolz calls this a valmap (https://toolz.readthedocs.io/en/latest/api.html#toolz.dicttoolz.valmap), if you want a generic name
    """makes new dataframe by implementing a function"""
    # CC: Better: Apply function to all values?
    dict_result = {}
    for wave in waves:
        dict_result[wave] = function(df_dict[wave])
    return dict_result

In [63]:
def select_spec_wave_respondents(df, wave):
    """select participants of specific wave"""
    new_df = df[df['panelpat'].str.contains(wave)]
    return new_df

In [64]:
def rename_specific_features_set(df, dict):  # CC: Naming. And Avoid builtin names.
    """renaming features according to dictionaries from dictionaries_rename.py
    and filtering these features"""  # CC: then it should be called rename_and_filter_specific_features or similar
    df_renamed = df.rename(columns=dict)
    names = list(dict.values())
    X_set = df_renamed[df_renamed.columns.intersection(names)]  # CC: .filter
    return X_set  # CC: Why not just df.filter(items=dict).rename(columns=dict)?

applying function to df, getting dict of dataframes for waves 1-6

In [9]:
dict_of_dfs = dict()  # CC: Naming.
for wave in waves:
    dict_of_dfs[wave] = select_spec_wave_respondents(df, wave)

Filtering only wavewise questions and personal features

In [10]:
def filter_wave_questions(wave, df):
    """select only questions from specific wave + personal features"""
    
    # wave related questions
    df_w = df.filter(regex='w' + wave)
    # personal features (coded with 'sd' prefix)
    df_sd = df.filter(regex='(sd)')
    # id, popnum and age are personal features which are not coded with prefixes
    # 'w' or 'sd', hence picked manually
    df_id = df['id']
    # changing string binned value to integer 
    df.loc[df.age == '>= 70', 'age'] = 70  # CC: It exists?
    df_age = df['age']
    y = df['panelpat']
    df_popnum = df['popnum']
    new_df = pd.concat([df_w, df_sd, df_popnum, df_id, df_age, y], axis=1)
    return new_df

In [84]:
def find_columns(df, **kwargs):
    return df.filter(**kwargs).columns.tolist()


def find_column(df, **kwargs):
    columns = find_columns(df, **kwargs)
    if len(columns) != 1:
        raise ValueError(f'Expected one column but found: {columns}')
    return columns[0]

In [89]:
# collect names
wave = '1'

wave_related_questions = find_columns(df, regex=f'w{wave}')
personal_features = find_columns(df, regex='(sd)')
additional_columns = ['popnum', 'id', 'age', 'panelpat']
relevant_columns = wave_related_questions + personal_features + additional_columns

df[relevant_columns]

In [11]:
for wave in waves:
    dict_of_dfs[wave] = filter_wave_questions(wave, dict_of_dfs[wave])

dropping outliers of interview duration column: who made it in less than 300 seconds

In [12]:
def drop_intdur_outliers(df):
    """drop rows with duration of the interview of less than 5 mins"""
    intdur = df.filter(like='intdur', axis=1).columns
    df = df[df[intdur[0]]>=300]
    return df

In [13]:
dict_of_dfs = apply_modification(drop_intdur_outliers, dict_of_dfs)

### Rate of *don't know* answers

We leave this function here because it needs original values preserved (although logically belongs to **engineered features** part)

In [14]:
def dont_know_ratio(df):
    """77, 88 values are either don't know/don't know party/don't know the person
    :output: the df with ratio column of such responses"""
    df['dont_know_ratio'] = ((df == 77).sum(axis=1) + (df == 88).sum(axis=1))/df.shape[1]
    # CC: In my opinion the denominator is wrong.
    return df

dict_of_dfs = apply_modification(dont_know_ratio, dict_of_dfs)

## Ordinal Features

for those which need to be ordinally encoded

In [15]:
def prepare_ordinals_to_transform(df):
    """rename ordinal columns that need to be coded (e.g. opinion questions), 
    throw other features, check all the unique values of these columns, print (disabled by #),
    replacing values with numbers, NaN values replaced by mode"""

    X_ordinal = rename_specific_features_set(df, get_ordinal_names())
    
    # 77, 88 values are "don't know" and 99 is refused, 12 is "would vote invalid"
    # so we make these values NaN for it not to be considered as ordinal values
    X_ordinal = X_ordinal.replace(dict.fromkeys([77, 88, 99, 12], np.NaN))

    # replacing NaN by mode  # CC: May call it "imputation".
    for column in X_ordinal.columns:
        X_ordinal[column].fillna(X_ordinal[column].mode()[0], inplace=True)
    # excluding string responses (like open questions)
    # X_ordinal = X_ordinal.select_dtypes(exclude=[object])
    drop_list = list(get_ordinal_names().keys())
    df = df.drop(columns=drop_list, errors='ignore')
    df = pd.concat([df, X_ordinal], axis=1)
    return df


dict_of_dfs = apply_modification(prepare_ordinals_to_transform, dict_of_dfs)

In [97]:
ordinal_names = get_ordinal_names()
# ordinal_names

In [100]:
q = rename_specific_features_set(df, ordinal_names)
q.head()

,age_group -sd2x2,HOUSEHOLD SIZE -sd5,MEMBERS OF HOUSEHOLD YOUNGER THAN 18 YEARS -sd6,HIGHEST LEVEL OF EDUCATION -sd7,ATTENDANCE OF RELIGIOUS SERVICES -sd9,INCOME SITUATION -sd22,NET HOUSEHOLD INCOME -sd23,popnum order,INTERVIEW DURATION IN SECONDS -w1_intdur,INTERVIEWDATUM (DAYS TO ELECTION DAY) -w1_dte,...,LEFT-RIGHT SELF-PLACEMENT -w6f_q13,DEGREE OF CLOSENESS TO PARTY -w6f_q17,TURNOUT: NATIONAL ELECTION 2017 -w6f_q18,MUSLIMS HAVE FEWER CHANCES IN AUSTRIA -w6f_q27x1,MUSLIMS ARE RARELY DISCRIMINATED AGAINST IN AUSTRIA -w6f_q27x2,ANGRY WHEN MUSLIMS ARE DISCRIMINATED AGAINST BECAUSE OF BELIEFS -w6f_q27x3,FEELING LIKE A STRANGER DUE TO THE MANY MUSLIMS -w6f_q27x4,EUROPEAN AND MUSLIM LIFESTYLE ARE EASILY COMPATIBLE -w6f_q27x5,INFLUENCE OF POLITICS ON ECONOMIC DEVELOPMENT -w6f_q39,QUIZ: USE OF RESOURCES -w6f_q40
0,1.0,3.0,1.0,4.0,6.0,2.0,99.0,3.0,12745.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,2.0,0.0,11.0,7.0,2.0,14.0,3.0,2322.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,2.0,0.0,10.0,3.0,2.0,12.0,3.0,815.0,129.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4.0,3.0,6.0,7.0,3.0,99.0,3.0,1362.0,129.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,1.0,0.0,14.0,5.0,3.0,14.0,3.0,910.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
q.dtypes.value_counts()

float64    698
dtype: int64

In [111]:
map_non_ordinal_responds_to_nan = dict.fromkeys([77, 88, 99, 12], np.NaN)
map_non_ordinal_responds_to_nan
qq = q.replace(to_replace=[77, 88, 99, 12], value=np.NaN)
qq.head(3)

,age_group -sd2x2,HOUSEHOLD SIZE -sd5,MEMBERS OF HOUSEHOLD YOUNGER THAN 18 YEARS -sd6,HIGHEST LEVEL OF EDUCATION -sd7,ATTENDANCE OF RELIGIOUS SERVICES -sd9,INCOME SITUATION -sd22,NET HOUSEHOLD INCOME -sd23,popnum order,INTERVIEW DURATION IN SECONDS -w1_intdur,INTERVIEWDATUM (DAYS TO ELECTION DAY) -w1_dte,...,LEFT-RIGHT SELF-PLACEMENT -w6f_q13,DEGREE OF CLOSENESS TO PARTY -w6f_q17,TURNOUT: NATIONAL ELECTION 2017 -w6f_q18,MUSLIMS HAVE FEWER CHANCES IN AUSTRIA -w6f_q27x1,MUSLIMS ARE RARELY DISCRIMINATED AGAINST IN AUSTRIA -w6f_q27x2,ANGRY WHEN MUSLIMS ARE DISCRIMINATED AGAINST BECAUSE OF BELIEFS -w6f_q27x3,FEELING LIKE A STRANGER DUE TO THE MANY MUSLIMS -w6f_q27x4,EUROPEAN AND MUSLIM LIFESTYLE ARE EASILY COMPATIBLE -w6f_q27x5,INFLUENCE OF POLITICS ON ECONOMIC DEVELOPMENT -w6f_q39,QUIZ: USE OF RESOURCES -w6f_q40
0,1.0,3.0,1.0,4.0,6.0,2.0,NaN,3.0,12745.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,2.0,0.0,11.0,7.0,2.0,14.0,3.0,2322.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,2.0,0.0,10.0,3.0,2.0,NaN,3.0,815.0,129.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for column in qq:
    a = 
    qq[column] = qq[column].fillna()

In [115]:
qq.columns

Index(['age_group -sd2x2', 'HOUSEHOLD SIZE -sd5',
       'MEMBERS OF HOUSEHOLD YOUNGER THAN 18 YEARS -sd6',
       'HIGHEST LEVEL OF EDUCATION -sd7',
       'ATTENDANCE OF RELIGIOUS SERVICES -sd9', 'INCOME SITUATION -sd22',
       'NET HOUSEHOLD INCOME -sd23', 'popnum order',
       'INTERVIEW DURATION IN SECONDS -w1_intdur',
       'INTERVIEWDATUM (DAYS TO ELECTION DAY) -w1_dte',
       ...
       'LEFT-RIGHT SELF-PLACEMENT -w6f_q13',
       'DEGREE OF CLOSENESS TO PARTY -w6f_q17',
       'TURNOUT: NATIONAL ELECTION 2017 -w6f_q18',
       'MUSLIMS HAVE FEWER CHANCES IN AUSTRIA -w6f_q27x1',
       'MUSLIMS ARE RARELY DISCRIMINATED AGAINST IN AUSTRIA -w6f_q27x2',
       'ANGRY WHEN MUSLIMS ARE DISCRIMINATED AGAINST BECAUSE OF BELIEFS -w6f_q27x3',
       'FEELING LIKE A STRANGER DUE TO THE MANY MUSLIMS -w6f_q27x4',
       'EUROPEAN AND MUSLIM LIFESTYLE ARE EASILY COMPATIBLE -w6f_q27x5',
       'INFLUENCE OF POLITICS ON ECONOMIC DEVELOPMENT -w6f_q39',
       'QUIZ: USE OF RESOURCES -w6f_q

In [117]:
qq['age_group -sd2x2'].mode()[0]

4.0

In [113]:
df.fillna?

## Dummy features

In [16]:
def prepare_dummies(df):
    """renaming dummy features, throw others, code as dummies, throw nan columns"""
    
    X_dummies = rename_specific_features_set(df, get_dummies_names())

    pol_cols = X_dummies.columns
    X_dummies = pd.get_dummies(
        X_dummies, columns=pol_cols, dummy_na=True, prefix_sep='__')
    X_dummies = X_dummies[X_dummies.columns.drop(
        list(X_dummies.filter(regex='nan')))]
    drop_list = list(get_dummies_names().keys())
    df = df.drop(columns=drop_list, errors='ignore')
    df = pd.concat([df, X_dummies], axis=1)
    return df

In [17]:
dict_of_dfs = apply_modification(prepare_dummies, dict_of_dfs)

## Binary features

Binary features processing (e.g. important issue: 1 if respondent mentioned the question, 0 otherwise).

In [18]:
def prepare_binary(df):
    """Rename binary features, remove other columns, replacing with numbers."""

    X_binary = rename_specific_features_set(df, get_binary_names())

    X_binary = X_binary.fillna(0)
    drop_list = list(get_binary_names().keys())
    df = df.drop(columns=drop_list, errors='ignore')
    df = pd.concat([df, X_binary], axis=1)
    return df

In [19]:
dict_of_dfs = apply_modification(prepare_binary, dict_of_dfs)

## Engineering new features

### Knowledge of political process

whether person knows voting age (1) or not (0), whether person knows PARLIAMENTARY THRESHOLD (1) or not (0)

In [20]:
def count_voting_age_awareness(df):
    """coded as binary feature depending on whether answer is correct"""
    
    age_column = df.filter(regex='w1_q31|w4f_q55|w6f_q41').columns
    df[age_column] = df[age_column].replace([16], True)    
    # Age with capital letter because otherwise it's mixed with personal 
    # feature of age and gets to wrong dataset of personal features
    df = df.rename(columns={age_column[0]: 'voting_Age_awareness'})
    # replace wrong values and NaN by 0
    df['voting_Age_awareness'][df['voting_Age_awareness'] != True] = False
    return df

def count_parl_threshold_column(df):
    """coded as binary feature depending on whether answer is correct"""
    
    parl_threshold_column = df.filter(items=['w1_q32', 'w3_q47', 'w4f_q56', 'w6f_q42']).columns
    df[parl_threshold_column] = df[parl_threshold_column].replace(['4%'], 1)
    df = df.rename(columns={parl_threshold_column[0]: 'knows_parl_threshold'})
    # replace wrong values and NaN by 0
    df['knows_parl_threshold'][df['knows_parl_threshold'] != 1] = 0
    return df

True

In [21]:
# wave specific questions
for wave in ['1', '4', '6']:
    dict_of_dfs[wave] = count_voting_age_awareness(dict_of_dfs[wave])
    
for wave in ['1', '3', '4', '6']:
    dict_of_dfs[wave] = count_parl_threshold_column(dict_of_dfs[wave])

### Political interest of participant
Find out how politically interested/active a respondent is.

In [190]:
def political_interest(df):  # CC: Improve naming? Functions names often include verbs.
    """count sum of activities that show political interest of respondent"""
    # CC: Simplify.
    visited_facebook_page = df.filter(
        like='VISITED FACEBOOK', axis=1).sum(axis=1)  # CC: Empty in one example.
    spoke_to_party_worker = df.filter(like='TALKED', axis=1).sum(axis=1)
    sum_interest = pd.concat(
        [visited_facebook_page, spoke_to_party_worker], axis=1).sum(axis=1)
    df['political_interest'] = sum_interest
    #sum_interest = pd.DataFrame(sum_interest)
    #sum_interest.columns = ['political_interest']
    return df

In [23]:
dict_of_dfs = apply_modification(political_interest, dict_of_dfs)

### Correct left-right identification
Did participant correctly place the given parties on a politic (left-right) spectrum?

* ÖVP - right
* GRÜNE - left
* SPÖ - left
* FPÖ - right
* NEOS - centrism (excluded)
* TEAM STRONACH - right
* LIST PETER PILZ - left

Initial data are from 0 (left) to 10 (right). If a person assigns 0:3 to a right party or 7:10 to a left party or 0:2/8:10 to centrism party then it is considered as wrong answer and coded as 0. Otherwise it is 1. New variable **lr_placement_correct**: rate how many times person placed a party in a "correct" way described above

**correct_placement_bin**: bottom 25% from lr_placement_correct are 0, others are 1

In [24]:
def correct_placement(df):
    """get binary feature with 25% of people who were the most incorrect and others"""
    lr_placement = df.filter(like='LEFT-RIGHT PLACEMENT:', axis=1)  # CC: Empty in one example.
    rights = lr_placement.filter(regex='OEVP|FPOE|TEAM STRONACH')
    rights.replace([0, 1, 2, 3], 0, inplace=True)
    rights[rights != 0] = 1
    lefts = lr_placement.filter(regex='THE GREENS|SPOE|LIST PETER PILZ')
    lefts.replace([7, 8, 9, 10], 0, inplace=True)
    lefts[lefts != 0] = 1

    lr_placement_bin = pd.concat([rights, lefts], axis=1)
    lr_placement_correct = lr_placement_bin.sum(
        axis=1) / lr_placement_bin.shape[1]
    lr_placement_correct = pd.DataFrame(lr_placement_correct)
    lr_placement_correct.columns = ['lr_placement_correct']
    lr_placement_correct.fillna(0, inplace=True)
    df['lr_placement_correct'] = lr_placement_correct
    return df

In [25]:
dict_of_dfs = apply_modification(correct_placement, dict_of_dfs)

### Quality

**ONLY W1 and W2:** making CHECK QUESTIONs binary (1 for correct answer, 0 for incorrect answer)

In [26]:
def prepare_check_question(df):
    """coded as binary feature depending on whether answer is correct"""
    # CC: Naming. Simpify.
    check_question = df.loc[:, df.columns.isin(['w1_q27x5', 'w2_q24x5'])]
    check_question[check_question != 4] = 0 # 4 is 'somewhat disagree'  # CC: Does this work well with previous cleaning?
    check_question = check_question.replace(4, 1)
    check_question = pd.DataFrame(check_question)
    check_question.columns = ['check_question']
    return check_question

In [27]:
check_question_w1 = prepare_check_question(dict_of_dfs['1'])
check_question_w2 = prepare_check_question(dict_of_dfs['2'])

counting number of words in open questions

In [28]:
def open_q_number(df):
    """count sum of words in open questions"""
    df_open_q = df.filter(regex='w2_q51x5t|w4_q84x5t')
    df_open_q.columns = ['words_open_question']
    df_open_q['words_open_question'] = df_open_q['words_open_question'].str.split(
    ).str.len()
    df_open_q.fillna(0, inplace=True)
    return df_open_q

In [29]:
df_open_q_w2 = open_q_number(dict_of_dfs['2'])
df_open_q_w4 = open_q_number(dict_of_dfs['4'])

### Correct left-right identification of politicians

In [30]:
def know_politicians_ratio(df):
    """rate of how well they know politicians"""
    X_know_politicians_ratio = df.loc[:, df.columns.isin(['KNOWLEDGE: HANS-PETER DOSKOZIL -w1_q33x1_1.0',
                                                                        'KNOWLEDGE: SOPHIE KARMASIN -w1_q33x2_2.0',  # not sure
                                                                        'KNOWLEDGE: SONJA HAMMERSCHMID -w1_q33x3_1.0',
                                                                        'KNOWLEDGE: HERBERT KICKL -w1_q33x4_3.0',
                                                                        'KNOWLEDGE: HANS-PETER DOSKOZIL -w4f_q57x1_1.0',
                                                                        'KNOWLEDGE: SOPHIE KARMASIN -w4f_q57x2_2.0',
                                                                        'KNOWLEDGE: SONJA HAMMERSCHMID -w4f_q57x3_1.0',
                                                                        'KNOWLEDGE: HERBERT KICKL -w4f_q57x4_3.0',
                                                                        'KNOWLEDGE: HANS-PETER DOSKOZIL -w6f_q43x1_1.0',
                                                                        'KNOWLEDGE: SOPHIE KARMASIN -w6f_q43x2_2.0',
                                                                        'KNOWLEDGE: SONJA HAMMERSCHMID -w6f_q43x3_1.0',
                                                                        'KNOWLEDGE: HERBERT KICKL -w6f_q43x4_3.0'])].sum(axis=1) / 4

    df['know_politicians_ratio'] = X_know_politicians_ratio
    return df

In [31]:
dict_of_dfs = apply_modification(
    know_politicians_ratio, dict_of_dfs)

In [196]:
# CC: Maybe to already at the beginning?
def add_label(df):
    """add 'OPINION: ' label to opinion related questions 
    (e.g. strongly agree/completely agree) for easier filtering """
    opinion_cols = df.filter(regex='|'.join(opinion_questions), axis=1).columns
    for col in opinion_cols:
        df.rename(columns={col: 'OPINION: ' + col}, inplace=True)
    return df

In [193]:
dict_of_dfs = apply_modification(add_label, dict_of_dfs)

In [34]:
# agree/disagree with some opinion
def same_response_rate(df):
    """check for straighlining (rate of opinion questions answered with the same option)"""

    X_mode_agreement = df.filter(like='OPINION', axis=1)
    same_agree_resp_rate = X_mode_agreement.stack().groupby(
        level=0).value_counts().max(level=0) / X_mode_agreement.shape[1]  # CC: Bit hard to read.
    df['same_agree_resp'] = same_agree_resp_rate
    return df

In [35]:
dict_of_dfs = apply_modification(same_response_rate, dict_of_dfs)

### (In)consistent answers

We filter out a few very similar (or diametrically opposed) questions and check if a participant gave similar (or opposed) answers

Manually, we selected the following questions.
* PREFER INDEPENDENT CITIZEN INSTEAD OF A PARTY MEMBER -w1_q27x8	
* THE PEOPLE SHOULD TAKE MOST IMPORTANT DECISIONS, NOT POLITICIANS -w1_q27x7
* 'FEELING LIKE A STRANGER DUE TO THE MANY MUSLIMS -w2_q21x4',
* 'EUROPEAN AND MUSLIM LIFESTYLE ARE EASILY COMPATIBLE -w2_q21x5',
* PEOPLE LIKE ME HAVE RECEIVED LESS THAN THEY DESERVE -w3_q35x1	
* PEOPLE LIKE ME GET LESS ATTENTION THAN OTHERS -w3_q35x2
* 'SAME ACCESS TO SOCIAL BENEFITS: ASYLUM SEEKERS -w4_q65x2',
* 'SAME ACCESS TO SOCIAL BENEFITS: NON-AUSTRIANS -w4_q65x1',
* IMMIGRANTS GET MORE ATTENTION -w5_q30x2	
* IMMIGRANTS HAVE RECEIVED MORE THAN THEY DESERVE -w5_q30x1
* OPINION: MOST POLITICIANS ARE TRUSTWORTHY -w6_q34x3,
* OPINION: POLITICIANS DO NOT CARE ABOUT WHAT PEOPLE LIKE ME THINK -w6_q34x5

This feature will be __$1$ if an answer is inconsistent__
and $0$ otherwise

In [36]:
def consistency_check(df):
    """Filter some similar (or diametrically opposed) questions 
    and check if a participant gave similar (or opposed) answeres"""
    positive_corr_opinion = df.loc[:, df.columns.isin(
        ['OPINION: PREFER INDEPENDENT CITIZEN INSTEAD OF A PARTY MEMBER -w1_q27x8',
         'OPINION: THE PEOPLE SHOULD TAKE MOST IMPORTANT DECISIONS, NOT POLITICIANS -w1_q27x7',
         'OPINION: PEOPLE LIKE ME HAVE RECEIVED LESS THAN THEY DESERVE -w3_q35x1',
         'OPINION: PEOPLE LIKE ME GET LESS ATTENTION THAN OTHERS -w3_q35x2',
         'OPINION: IMMIGRANTS GET MORE ATTENTION -w5_q30x2',
         'OPINION: IMMIGRANTS HAVE RECEIVED MORE THAN THEY DESERVE -w5_q30x1',
         'OPINION: SAME ACCESS TO SOCIAL BENEFITS: ASYLUM SEEKERS -w4_q65x2',
         'OPINION: SAME ACCESS TO SOCIAL BENEFITS: NON-AUSTRIANS -w4_q65x1',
         'OPINION: FEELING LIKE A STRANGER DUE TO THE MANY MUSLIMS -w2_q21x4',
         'OPINION: EUROPEAN AND MUSLIM LIFESTYLE ARE EASILY COMPATIBLE -w2_q21x5',
         'OPINION: MOST POLITICIANS ARE TRUSTWORTHY -w6_q34x3',
         'OPINION: POLITICIANS DO NOT CARE ABOUT WHAT PEOPLE LIKE ME THINK -w6_q34x5'])]
    positive_corr_opinion.replace([0, 1], 0, inplace=True)  # disagree
    positive_corr_opinion.replace([4, 5], 1, inplace=True)  # agree
    positive_corr_opinion.replace([2, 3], np.NaN, inplace=True)
    inconsistency = positive_corr_opinion[positive_corr_opinion.columns[0]].eq(
        positive_corr_opinion[positive_corr_opinion.columns[1]]).astype(int)
    inconsistency = abs(inconsistency-1)

    df['inconsistency'] = inconsistency
    return df

In [37]:
dict_of_dfs = apply_modification(consistency_check, dict_of_dfs)
# in waves 2 and 6 questions are quite opposite in terms of inconsistency
dict_of_dfs['2']['inconsistency'] = abs(dict_of_dfs['2']['inconsistency']-1)
dict_of_dfs['6']['inconsistency'] = abs(dict_of_dfs['6']['inconsistency']-1)

In [38]:
def response_hour(df):
    """function returns columns of which time of the day the response 
    was given and binary feature of day of the week (weekday/weekend)"""
    timestamp = df.filter(like='_date', axis=1).astype('datetime64[ns]')  # CC: Defer to csv reader
    timestamp.columns = ['timestamp_colname']
    hour = timestamp.timestamp_colname.dt.hour
    hour = pd.DataFrame(hour)
    hour.columns = ['timeOfResponding']  # CC: Naming. Don't mix casing
    nighttime = list([23, 0, 1, 2, 3, 4, 5])
    morningtime = list(range(6, 9))
    worktime = list(range(9, 17))
    eveningtime = list(range(17, 23))
    # CC: Needs explanation. Okay to do but should be motivated.
    hour.replace(nighttime, 'nighttime', inplace=True)
    hour.replace(morningtime, 'morning', inplace=True)
    hour.replace(worktime, '9-5', inplace=True)
    hour.replace(eveningtime, 'evening', inplace=True)
    hour_dummies = pd.get_dummies(hour, dummy_na=True, prefix_sep='_')  # CC: Naming. Not an hour dummy...

    timestamp['weekendResponse'] = timestamp['timestamp_colname'].dt.day_name()
    timestamp['weekendResponse'].replace(
        ['Saturday', 'Sunday'], 1, inplace=True)
    timestamp['weekendResponse'][timestamp['weekendResponse'] != 1] = 0
    time_vars = pd.concat([timestamp['weekendResponse'], hour_dummies], axis=1)
    df = pd.concat([df, time_vars], axis=1)
    return df

In [39]:
dict_of_dfs = apply_modification(response_hour, dict_of_dfs)

### Days to respond (`dte`)
We noticed that the `dte` (="days to election") feature show up significantly in the model predicting attrition. For easier interpretation we "normalized" this feature counting how many days after the first respondent a participant completed the wave.

In [40]:
def days_to_respond(df):
    """how many days it took to respond assuming all the samples got the survey at the same time,
    starting point was from the very first response registered"""
    # CC: I don't understand this one. I would compute it differently.
    df = df.loc[:,~df.columns.duplicated()]  # CC: Did not exist in one example.
    dtes = (df.filter(like='dte', axis=1)).to_numpy()
    rankdata_list = rankdata(list(map(lambda x: -x, dtes)), method='dense')  # CC: If that, then why not just rankdata(-dtes, method='dense')?
    rankdata_df = pd.DataFrame(rankdata_list)
    rankdata_df.columns = ['days_to_respond']
    rankdata_df.replace(list(range(9, 17)), 9, inplace=True)  # CC: Magic numbers. Why range(9, 17)? If this is clipping, then clip it.
    rankdata_df.index = df.index
    df['days_to_respond'] = rankdata_df
    return df

In [41]:
dict_of_dfs = apply_modification(days_to_respond, dict_of_dfs)

### Participation

In [42]:
def participated_only_once(df):
    """binary feature: checking if person participated only once 
    (since there are only 3 values we filter that manually)"""
    # CC: Is there a feature giving "participation count"?
    # Why only-once and not only-once-or-twice or ...?
    # CC: If count should be included, could do df['panelpat'].apply(lambda s: s.count('.'))
    df['participated_only_once'] = df['panelpat'].isin(['1.....', '.....6', '...4..'])
    return df

dict_of_dfs = apply_modification(participated_only_once, dict_of_dfs)

In [43]:
def refreshment_respondent(df):
    """Feature indicating whether person joined survey after wave 3 (code: True) or not (code: False)"""
    
    df['refreshment'] = ~df['panelpat'].str.contains('1|2|3')
    return df

dict_of_dfs = apply_modification(refreshment_respondent, dict_of_dfs)

In [44]:
def came_back(wave, df):
    """if participant has dropped the survey and then came back in a later wave, 
    the value equals to the number of the last wave one has participated before the current one, 
    otherwise if one has never dropped or never came back: 0"""
    if wave == '1' or wave == '2':
        return df
    else:
        panelpat_df = pd.DataFrame(df['panelpat'])
        panelpat_df['panelpat'] = panelpat_df['panelpat'].str.replace('.', '0')
        panelpat_df[['previous_waves', 'future_waves']] = panelpat_df['panelpat'].str.split(
            '0' + str(wave), expand=True)
        # participant can miss 2 waves in a row (or more), therefore we delete duplicate 0 to get the latest wave of participation
        panelpat_df.previous_waves = panelpat_df.previous_waves.apply(
            lambda w: "".join(sorted(set(w))))
        df['came_back_from'] = [w.strip()[-1]
                                for w in panelpat_df['previous_waves']]
        # if one has not dropped in previous wave we also change is to 0
        df.loc[df['came_back_from'] >= wave, 'came_back_from'] = 0
        return df

In [45]:
#for wave in waves:
#    dict_of_dfs[wave] = came_back(wave, dict_of_dfs[wave])

### Did they drop before?
New variable: whether participant dropped the survey at least in one of previous wave:
        * 1 if dropped before
        * 0 if participated in all waves

In [46]:
# respondents recrouted later are considered as those who never dropped 
# ('...456': 186 samples and '.....6': 117 samples)
def dropped_before(wave, df):
    """split panelpat into 2 parts, number of wave is delimeter, 
    get previous and future waves of participation (in terms of current wave)
    then check if there are missings in previous waves,
    if so, then coded as 1 (dropped before), otherwise 0"""
    panelpat_df = pd.DataFrame(df['panelpat'])
    panelpat_df[panelpat_df == '...456|.....6'] = 0
    panelpat_df[['previous_waves', 'future_waves']
                ] = panelpat_df['panelpat'].str.split(wave, expand=True)
    whether_dropped_before = panelpat_df['previous_waves']
    whether_dropped_before = whether_dropped_before.str.contains(
        '.', regex=False)
    df['whether_dropped_before'] = whether_dropped_before

    return df

In [47]:
for wave in waves:
    dict_of_dfs[wave] = dropped_before(wave, dict_of_dfs[wave])

## Dependent variable

* transforming 'panelpat' into classes: 
    * 0 for people who dropped
    * 1 for respondents who stayed
* making y series (which is panelpat)

In [48]:
def transform_panelpat(wave, df):
    """coding panelpat as follows: if person participated in next wave, then: 1, otherwise: 0, 
    extract dependent variable for concatenating at the end with all the prepared features"""
    i = int(wave)
    i += 1
    i = str(i)  # CC: Why not one line?
    df['panelpat'] = df['panelpat'].str.contains(i).astype(int)
    return df

In [49]:
for wave in ['1', '2', '3', '4', '5']:
    dict_of_dfs[wave] = transform_panelpat(wave, dict_of_dfs[wave])

## Splitting into political and personal datasets

concatenating wave specific columns:

In [50]:
def list_for_splitting(data_all):  # CC: Naming. What is the return value about?
    """used to split df into political and personal features"""
    # CC: Namings.
    ohe_features = ['HOUSEHOLD SIZE -sd5',
                    'MEMBERS OF HOUSEHOLD YOUNGER THAN 18 YEARS -sd6',
                    'HIGHEST LEVEL OF EDUCATION -sd7',
                    'ATTENDANCE OF RELIGIOUS SERVICES -sd9',
                    'JOB SITUATION -sd11',
                    'INCOME SITUATION -sd22',
                    'NET HOUSEHOLD INCOME -sd23',
                    'DESCRIPTION OF RESIDENTIAL AREA -sd24',
                    'ADDITIONAL OCCUPATION -sd13',
                    'RELIGIOUS AFFILIATION -sd8',
                    'Country of birth, repondent -sd18',
                    'Country of birth, mother -sd19',
                    'Country of birth, father -sd20',
                    'GENDER -sd3',
                    'age',
                    'popnum',
                    'FEDERAL STATE -sd4',
                    'CURRENT PERSONAL SITUATION -sd10',
                    'OTHER OCCUPATION -sd12',
                    'TYPE OF OCCUPATION -sd14',
                    'PREVIOUS TYPE OF OCCUPATION -sd16',
                    'AUSTRIAN CITIZENSHIP FROM BIRTH -sd17',
                    'UNION MEMBERSHIP -sd21',
                    'EVER EMPLOYMENT -sd15',
                    'came_back_from']
    transformed_features_names = []
    for i in ohe_features:
        a = list(data_all.filter(like=i, axis=1))
        transformed_features_names.append(a)

    list_personals = []
    for i in list(range(0, len(transformed_features_names))):
        for j in list(range(0, len(transformed_features_names[i]))):
            b = transformed_features_names[i][j]
            list_personals.append(b)
    return list_personals

In [51]:
dict_of_dfs['1'] = pd.concat([dict_of_dfs['1'],
                              check_question_w1], axis=1)

dict_of_dfs['2'] = pd.concat([dict_of_dfs['2'],
                              check_question_w2,
                              df_open_q_w2], axis=1)

dict_of_dfs['4'] = pd.concat([dict_of_dfs['4'],
                              df_open_q_w4,
                              ], axis=1)

In [52]:
X_personal = {}
X_political = {}
for wave in waves:
    list_personals = list_for_splitting(dict_of_dfs[wave])
    X_personal[wave] = dict_of_dfs[wave][list_personals]
    X_political[wave] = dict_of_dfs[wave].drop(list_personals, axis=1)
    # some id values are missing but only 1 per wave, so we impute index there
    X_political[wave].interpolate(inplace=True)
    # drop redundant columns (such as open questions because we 
    # already added its prepared versions), also
    # columns with NaN and others like those which we did not use (e.g. w1_panelist)
    redundant_columns = ['w1_panelist', 'w1_weightd', 'w1_weightp', 'w1_date', 'sd1',
                         'w2_panelist', 'w2_weightd', 'w2_weightp', 'w2_date', 'w2_q45x1t_id',
                         'w2_q45x2t_id', 'w3_panelist', 'w3_weightd', 'w3_weightp', 'w3_date',
                         'w4_panelist', 'w4_weightd', 'w4_weightp', 'w4_date', 'w4f_q56t', 'w4_q62t',
                         'w4_q63t', 'w4_q64t', 'w4_q78x1t_id', 'w4_q78x2t_id', 'w4_q80x5t', 'w4_q84x5t',
                         'w5_panelist', 'w5_weightd', 'w5_weightp', 'w5_date',
                         'w6_panelist', 'w6_weightd', 'w6_weightp', 'w6_date', 'w6_exp1',
                         'w6_q31t', 'w6_q31dur', 'w6_q32t', 'w6_q33t', 'w6_q46x1t_id',
                         'w6_q46x2t_id', 'w6_q57split', 'w6f_q42t']
    X_political[wave] = X_political[wave].drop(
        columns=redundant_columns, errors='ignore')
    X_political[wave].dropna(axis=1, inplace=True)
    # filling age NaN by mean
    X_personal[wave] = X_personal[wave].fillna(
        value=X_personal[wave]['age'].mean())
# making dependent feature (panelpat) right-hand sided
waves = ['1', '2', '3', '4', '5']
for wave in waves:
    X_political[wave]['panelpat'] = X_political[wave].pop('panelpat')
    X_personal[wave]['panelpat'] = X_political[wave]['panelpat']
    X_personal[wave]['id'] = X_political[wave]['id']

## Some final checks

checking why for some datasets number of columns is different from previous version

In [53]:
waves = ['1', '2', '3', '4', '5', '6']
for wave in waves:
    print('Wave: ', wave)
    print('X_personal: ', X_personal[wave].shape)
    print('X_political: ', X_political[wave].shape)

Wave:  1
X_personal:  (4019, 103)
X_political:  (4019, 354)
Wave:  2
X_personal:  (3133, 103)
X_political:  (3133, 335)
Wave:  3
X_personal:  (2994, 103)
X_political:  (2994, 452)
Wave:  4
X_personal:  (3166, 103)
X_political:  (3166, 622)
Wave:  5
X_personal:  (3026, 103)
X_political:  (3026, 351)
Wave:  6
X_personal:  (2974, 101)
X_political:  (2974, 372)


check for NaN in all datasets and getting its index if relevant

In [54]:
for wave in waves:
    print('Wave: ', wave)
    print(X_personal[wave].isnull().values.any())
    print(X_personal[wave].loc[pd.isnull(
        X_personal[wave]).any(1), :].index.values)
    print(X_political[wave].isnull().values.any())
    print(X_political[wave].loc[pd.isnull(
        X_political[wave]).any(1), :].index.values)

Wave:  1
False
[]
False
[]
Wave:  2
False
[]
False
[]
Wave:  3
False
[]
False
[]
Wave:  4
False
[]
False
[]
Wave:  5
False
[]
False
[]
Wave:  6
False
[]
False
[]


## Saving as csv

In [55]:
for wave in waves:
    X_political[wave].to_csv(
        f'data/data_online_political_w{wave}.csv', index=False)  # CC: Maybe different container to preserve data types?
    X_personal[wave].to_csv(
        f'data/data_online_personal_w{wave}.csv', index=False)